# Image Management API - Test with Custom Mock User

This notebook allows testing the image API with a custom mock user, resolving referential integrity issues related to team_id.

In [12]:




# Import necessary libraries
import requests
import json
import uuid
import io
import os
from IPython.display import display, HTML
from PIL import Image as PILImage
import base64
import re

In [13]:
# API base configuration
BASE_URL = "http://localhost:8080"  # Adjust as needed
API_KEY = "imapi_demo_key_for_testing_purposes_only"  # Adjust as needed

# Configuration to replace the mock user
# IMPORTANT: Replace this team_id with one that exists in the database!
MOCK_CONFIG = {
    "username": "daniel.schulz_computer_vision_team",
    "email": "daniel.schulz_computer_vision_team@sereact-ai.com",
    "team_id": "2e07cc40-68e2-40cc-b686-cb05dce6c408",
    "id": "37c9e956-3105-4846-bc81-261eb6094d2c",
    "created_at": "2025-05-30T20:27:00.608435+00:00",
}

## 3. Support Functions for API Calls

In [14]:
def make_api_request(method, endpoint, headers=None, params=None, json_data=None, files=None, data=None):
    """Base function to make API requests, applying the mock user"""
    url = f"{BASE_URL}{endpoint}"
    
    # Prepare headers with authentication and x-mock-user
    if headers is None:
        headers = {}
    
    # Add API key if not present
    if "Authorization" not in headers:
        headers["Authorization"] = API_KEY
    
    # Add mock user configuration
    mock_header = base64.b64encode(json.dumps(MOCK_CONFIG).encode()).decode()
    headers["X-Mock-User"] = mock_header
    
    # If we're sending form data, explicitly add team_id
    if data is None:
        data = {}
    
    # Explicitly add team_id to the data
    data["team_id"] = MOCK_CONFIG["team_id"]
    
    print(f"Request: {method} {url}")
    print(f"Using mock user: {MOCK_CONFIG['username']} (Team ID: {MOCK_CONFIG['team_id']})")
    
    # Make the request
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        params=params,
        json=json_data,
        files=files,
        data=data
    )
    
    print(f"Status: {response.status_code}")
    
    # Try to parse the response as JSON
    try:
        result = response.json()
        print("Response:")
        print(json.dumps(result, indent=2)[:500])
        if len(json.dumps(result, indent=2)) > 500:
            print("... (truncated)")
        return result
    except:
        # Not JSON, return the text
        print(f"Response (not JSON):\n{response.text[:200]}")
        if len(response.text) > 200:
            print("... (truncated)")
        return response.text

## 5. Create Image Upload Function with Mock User

In [17]:
def create_test_image(color, width=300, height=200):
    """Creates a test image with solid color"""
    if color == "red":
        rgb = (255, 0, 0)
    elif color == "green":
        rgb = (0, 255, 0)
    elif color == "blue":
        rgb = (0, 0, 255)
    else:
        rgb = (128, 128, 128)  # gray
    
    # Create image
    img = PILImage.new('RGB', (width, height), rgb)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    img_byte_arr.seek(0)
    return img_byte_arr

def upload_image_with_mock_user(color="red", description="Test image", tags="test,mock,user"):
    """Uploads an image using the configured mock user"""
    # Create test image
    img_bytes = create_test_image(color)
    
    # Prepare file for upload
    files = {
        'file': (f"{color}_test.jpg", img_bytes.getvalue(), 'image/jpeg')
    }
    
    # Form data
    data = {
        'description': description,
        'tags': tags,
        'alt_text': f"A test image {color}",
        'team_id': MOCK_CONFIG['team_id']  # Important: explicitly include team_id
    }
    
    # Perform upload
    result = make_api_request("POST", "/api/v1/images", files=files, data=data)
    
    # Display the image if there's an access URL
    if isinstance(result, dict) and "access_url" in result:
        print("\nImage:")
        display(HTML(f'<img src="{result["access_url"]}" width="300" />'))
    
    return result

## 6. Update Mock User Configuration with Valid Team ID

Use this cell to update the mock user's `team_id` to a valid value from the database.

In [16]:
# Update MOCK_CONFIG with a valid team_id
# You can select one of the IDs listed above

# For example, if the first team in the list is valid:
if valid_teams:
    MOCK_CONFIG['team_id'] = valid_teams[0]['id']
    print(f"MOCK_CONFIG updated with team_id: {MOCK_CONFIG['team_id']}")
else:
    print("WARNING: No valid teams found. Configure the team_id manually.")
    # You can manually set a known team_id:
    # MOCK_CONFIG['team_id'] = "11111111-1111-1111-1111-111111111111"

# Display current configuration
print("\nCurrent mock user configuration:")
print(json.dumps(MOCK_CONFIG, indent=2))

NameError: name 'valid_teams' is not defined

## 7. Test Image Upload with Mock User

In [18]:
# Upload a red image
print("Uploading red image...")
red_result = upload_image_with_mock_user("red", "Red test image", "red,test,mock")

Uploading red image...
Request: POST http://localhost:8080/api/v1/images
Using mock user: daniel.schulz_computer_vision_team (Team ID: 2e07cc40-68e2-40cc-b686-cb05dce6c408)
Status: 500
Response:
{
  "detail": "Internal server error"
}


In [ ]:
# Upload a green image
print("Uploading green image...")
green_result = upload_image_with_mock_user("green", "Green test image", "green,test,mock")

In [ ]:
# Upload a blue image
print("Uploading blue image...")
blue_result = upload_image_with_mock_user("blue", "Blue test image", "blue,test,mock")

## 8. List Uploaded Images

In [ ]:
# List all images
print("Listing all images...")
all_images = make_api_request("GET", "/api/v1/images")

## 9. Summary and Next Steps

If the solution above doesn't work, it may be necessary to modify the `auth.py` file on the server to accept the `X-Mock-User` header. Refer to the example code in section 4 of this notebook.

Another alternative is to create a custom endpoint on the server that explicitly accepts a `user_id` and `team_id` for upload.